TEXT TO SPEECH

In [1]:
# !pip install pyttsx3
import pyttsx3
text_speech= pyttsx3.init()

ans="AI Chat is an AI chatbot that writes text. You can use it to write stories, messages, or programming code. You can use the AI chatbot as a virtual tutor in almost any subject."
ans = 'HELLLO '
text_speech.say(ans)
text_speech.runAndWait()


SPEECH RECOGNATION

In [2]:
# !pip install pyaudio
# !pip install SpeechRecognition
import speech_recognition as sr
r=sr.Recognizer()
def speak():
    r=sr.Recognizer()

    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        print("Say")
        audio=r.listen(source)
        try:
            print(r.recognize_google(audio,language='hi-In'))
            return r.recognize_google(audio)
        except Exception as e:
            print("error"+str(e))
            return "error"+str(e)

#speak()

PRE PROCESSING

In [3]:
import pandas as pd 
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [4]:
df=pd.read_csv("hindiQuestion.csv")
df=df.drop(["squad_id"],axis="columns")

# pattern = r'[^\d\s\u0900-\u097F]+'
df['question']


0        कार्ल हेनरिक उलरिच ने इस वर्गीकरण योजना को कब ...
1        एक जीव जो सर्केडियन लय का उपयोग करता है वह अपन...
2        किस टेनेसी शहर में अफ्रीकी-अमेरिकी आबादी सबसे ...
3                गौतम बुद्ध के प्रवचनों में क्या शामिल है?
4                        किस पार्टी का नानजिंग पर शासन है?
                               ...                        
69974                    शिखर सम्मेलनों की जलवायु कैसी है?
69975                             मैकडरमॉट को कब चुना गया?
69976    नया लिवरपूल एफसी स्टेडियम 1879 में कहाँ स्थित था?
69977    नमक के अलावा माली में और कौन-से बड़े प्राकृतिक...
69978    वानूआतू में किस देश से मिशनरियों ने प्रेसबिटेर...
Name: question, Length: 69979, dtype: object

In [5]:
pattern = r'[^\d\s\u0900-\u097F]+'
#trial
test = df["question"][0]+"! ? ; ::: + कार्ल ; हेनरिक उलरिच ने इस वर्गीकरण योजना को कब;; विकसित किया   "
print(test)
test = re.sub(pattern, '', test )
print(test)



##tokenize
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()
print(word_tokenize(test))
wnl.lemmatize(word_tokenize(test)[3])


#trial

कार्ल हेनरिक उलरिच ने इस वर्गीकरण योजना को कब विकसित किया?! ? ; ::: + कार्ल ; हेनरिक उलरिच ने इस वर्गीकरण योजना को कब;; विकसित किया   
कार्ल हेनरिक उलरिच ने इस वर्गीकरण योजना को कब विकसित किया     कार्ल  हेनरिक उलरिच ने इस वर्गीकरण योजना को कब विकसित किया   
['कार्ल', 'हेनरिक', 'उलरिच', 'ने', 'इस', 'वर्गीकरण', 'योजना', 'को', 'कब', 'विकसित', 'किया', 'कार्ल', 'हेनरिक', 'उलरिच', 'ने', 'इस', 'वर्गीकरण', 'योजना', 'को', 'कब', 'विकसित', 'किया']


'ने'

DATA PREPERATION FUNCTION

In [6]:

def data_prep(text):
    
    text = re.sub(r'[^\d\s\u0900-\u097F]+', ' ', text )
    text=" ".join(t for t in text.split(" ") if t not in hindi_stopwords)
    return text
hindi_stopwords = open('final_stopwords.txt','r', encoding="utf8")
hindi_stopwords=(hindi_stopwords.read()).split('\n')
df["question"] = df["question"].apply(data_prep)


tf=TfidfVectorizer()
tf_train=tf.fit_transform(df["question"])
df_check=pd.DataFrame(tf_train.toarray(),columns=tf.get_feature_names_out())

le= LabelEncoder()
df["Answers_Code"]=le.fit_transform(df["answer"])
df



,Unnamed: 0,id,answer,context,question,Answers_Code
0,0,1,१८६० के दशक में,1860 के दशक में कार्ल हेनरिक उलरिच द्वारा निजी...,कार्ल हेनरिक उलरिच वर्गीकरण योजना विकसित,50801
1,1,2,संसाधन,इस प्रकार वे जीवों को उन लोगों की तुलना में पर...,जीव सर्केडियन लय उपयोग लाभ के उपयोग नहीं,45498
2,2,3,मेम्फिस,लोकतांत्रिक ताकत ज्यादातर चार प्रमुख शहरों के ...,टेनेसी शहर अफ्रीकी अमेरिकी आबादी सबसे अधिक,37361
3,3,4,सुता पिटक,""""" ""प्यारेली त्रिपिटक"" "", जिसका अर्थ"" ""तीन टोक...",गौतम बुद्ध के प्रवचनों शामिल,47799
4,4,5,सीपीसी,"""वर्तमान में, नानजिंग की सरकार का पूरा नाम"" ""न...",पार्टी नानजिंग शासन,47693
...,...,...,...,...,...,...
69974,69974,69975,कोई भी शिखर स्थायी बर्फ के क्षेत्र में नहीं पह...,कोई भी शिखर स्थायी बर्फ के क्षेत्र में नहीं पह...,शिखर सम्मेलनों जलवायु कैसी,17812
69975,69975,69976,1988,"संघीय रूप से, सिएटल वाशिंगटन के 7 वें कांग्रेस...",मैकडरमॉट चुना,3088
69976,69976,69977,स्टेनली पार्क,एवर्टन मूल रूप से स्टेनली पार्क के दक्षिण-पूर्...,नया लिवरपूल एफसी स्टेडियम 1879 स्थित,48917
69977,69977,69978,प्रमुख प्राकृतिक संसाधनों में सोना शामिल है,माली के प्रमुख प्राकृतिक संसाधनों में से कुछ म...,नमक के अलावा माली प्राकृतिक संसाधन,31226


In [7]:
# from sklearn.naive_bayes import MultinomialNB
# mn=MultinomialNB()
# mn.fit(df_check,df["Answers_Code"])

In [8]:
 import pickle
# pickle.dump(mn , open("/Agastya_Hindi/trainedModel",'wb'))


#fetching trained model
mn = pickle.load( open("/Agastya_Hindi/trainedModel",'rb'))

In [9]:

userInput=['टेनेसी शहर']
testing=tf.transform(userInput)
res=mn.predict(testing)
Ans=df["context"].unique()
Ans=Ans.tolist()
Ans_Code=df["Answers_Code"].unique()
Ans_Code=Ans_Code.tolist()
print ('\n\n\n')
for i in res:
    print(Ans[Ans_Code.index(i)])

c:\Users\DeskTop\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(






वास्तविक स्वतंत्रता की ओर किसी भी कदम को 1956 की हंगरी क्रांति और 1968 की प्राग बसंत में सैन्य बल द्वारा दबा दिया गया था।
